<a href="https://colab.research.google.com/gist/maclandrol/3fb46061789b5414445e2992988a4dab/02_Python_Outils_Medicaux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 02. Python : Outils pour l'Analyse Médicale

**Enseignant:** Emmanuel Noutahi, PhD

---

**Objectif:** Analyser un vrai dataset médical pour prédire les maladies cardiaques.

**Ce que vous allez apprendre :**
- Charger et explorer des données médicales réelles
- Identifier les facteurs de risque cardiovasculaire
- Créer des visualisations médicales
- Construire un modèle prédictif simple

**Dataset:** Cleveland Heart Disease (303 patients, 14 caractéristiques cliniques)

**Important:** Vous apprenez à utiliser des outils d'analyse, pas à devenir programmeur.

## Configuration et Installation

In [ ]:
# Installation des bibliothèques nécessaires pour Google Colab
!pip install pandas numpy matplotlib seaborn scikit-learn -q

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("✅ Bibliothèques installées et configurées avec succès")

## Chargement des Données

In [ ]:
# Charger le dataset Heart Disease depuis UCI Repository
# URL directe vers les données
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'

# Noms des colonnes selon la documentation UCI
column_names = [
    'age',        # Âge
    'sex',        # Sexe (1 = masculin, 0 = féminin)
    'cp',         # Type de douleur thoracique
    'trestbps',   # Pression artérielle au repos
    'chol',       # Cholestérol sérique
    'fbs',        # Glycémie à jeun > 120 mg/dl
    'restecg',    # Résultats ECG au repos
    'thalach',    # Fréquence cardiaque maximale
    'exang',      # Angine induite par l'exercice
    'oldpeak',    # Dépression ST
    'slope',      # Pente du segment ST
    'ca',         # Nombre de vaisseaux colorés
    'thal',       # Thalassémie
    'target'      # Diagnostic (0 = pas de maladie, >0 = maladie)
]

try:
    # Charger les données
    df = pd.read_csv(url, names=column_names, na_values='?')
    
    # Convertir la cible en binaire (0 = pas de maladie, 1 = maladie)
    df['target'] = (df['target'] > 0).astype(int)
    
    print("✅ Dataset chargé avec succès depuis UCI Repository")
    
except Exception as e:
    print(f"⚠️ Erreur lors du chargement: {e}")
    print("Création d'un dataset d'exemple...")
    
    # Dataset de secours
    np.random.seed(42)
    n_samples = 303
    
    df = pd.DataFrame({
        'age': np.random.randint(29, 77, n_samples),
        'sex': np.random.choice([0, 1], n_samples),
        'cp': np.random.choice([0, 1, 2, 3], n_samples),
        'trestbps': np.random.randint(94, 200, n_samples),
        'chol': np.random.randint(126, 564, n_samples),
        'fbs': np.random.choice([0, 1], n_samples, p=[0.85, 0.15]),
        'restecg': np.random.choice([0, 1, 2], n_samples),
        'thalach': np.random.randint(71, 202, n_samples),
        'exang': np.random.choice([0, 1], n_samples),
        'oldpeak': np.random.uniform(0, 6.2, n_samples),
        'slope': np.random.choice([0, 1, 2], n_samples),
        'ca': np.random.choice([0, 1, 2, 3], n_samples),
        'thal': np.random.choice([0, 1, 2, 3], n_samples),
        'target': np.random.choice([0, 1], n_samples)
    })

print(f"📊 Dataset chargé: {df.shape[0]} patients, {df.shape[1]} variables")
print(f"📝 Premières lignes:")
df.head()

## 📋 Exploration des Données

In [ ]:
# Définir les descriptions des variables en français
descriptions = {
    'age': 'Âge (années)',
    'sex': 'Sexe (0=Femme, 1=Homme)',
    'cp': 'Type douleur thoracique (0-3)',
    'trestbps': 'Pression artérielle repos (mmHg)',
    'chol': 'Cholestérol sérique (mg/dl)',
    'fbs': 'Glycémie à jeun >120 mg/dl (0=Non, 1=Oui)',
    'restecg': 'ECG au repos (0-2)',
    'thalach': 'Fréquence cardiaque max (bpm)',
    'exang': 'Angine d\'effort (0=Non, 1=Oui)',
    'oldpeak': 'Dépression ST (mm)',
    'slope': 'Pente segment ST (0-2)',
    'ca': 'Nb vaisseaux colorés (0-3)',
    'thal': 'Type thalassémie (0-3)',
    'target': 'Maladie cardiaque (0=Non, 1=Oui)'
}

print("🏥 VARIABLES CLINIQUES:")
print("=" * 50)
for var, desc in descriptions.items():
    print(f"{var:10} : {desc}")

In [ ]:
# Informations générales sur le dataset
print("📈 INFORMATIONS GÉNÉRALES:")
print("=" * 40)
print(f"• Nombre total de patients: {len(df)}")
print(f"• Nombre de variables: {df.shape[1]}")
print(f"• Patients avec maladie cardiaque: {df['target'].sum()} ({df['target'].mean()*100:.1f}%)")
print(f"• Patients sains: {(df['target']==0).sum()} ({(df['target']==0).mean()*100:.1f}%)")
print(f"• Valeurs manquantes: {df.isnull().sum().sum()}")

# Répartition par sexe
print("\n👥 RÉPARTITION PAR SEXE:")
sex_counts = df['sex'].value_counts()
print(f"• Femmes: {sex_counts[0]} ({sex_counts[0]/len(df)*100:.1f}%)")
print(f"• Hommes: {sex_counts[1]} ({sex_counts[1]/len(df)*100:.1f}%)")

# Statistiques d'âge
print("\n🎂 STATISTIQUES D'ÂGE:")
print(f"• Âge moyen: {df['age'].mean():.1f} ans")
print(f"• Âge médian: {df['age'].median():.1f} ans")
print(f"• Écart-type: {df['age'].std():.1f} ans")
print(f"• Âge min/max: {df['age'].min()}/{df['age'].max()} ans")

In [ ]:
# Statistiques descriptives complètes
print("📊 STATISTIQUES DESCRIPTIVES:")
print("=" * 50)
df.describe().round(2)

## 📊 Visualisations des Données

In [ ]:
# Vue d'ensemble des données
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Vue d\'Ensemble des Données Cardiovasculaires', fontsize=16, fontweight='bold')

# 1. Distribution de l'âge par diagnostic
axes[0,0].hist([df[df['target']==0]['age'], df[df['target']==1]['age']], 
               bins=15, alpha=0.7, 
               label=['Pas de maladie', 'Maladie cardiaque'],
               color=['lightblue', 'salmon'])
axes[0,0].set_title('Distribution de l\'Âge par Diagnostic')
axes[0,0].set_xlabel('Âge (années)')
axes[0,0].set_ylabel('Nombre de patients')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Pression artérielle vs Cholestérol
colors = df['target'].map({0: 'lightblue', 1: 'salmon'})
axes[0,1].scatter(df['trestbps'], df['chol'], c=colors, alpha=0.6, s=50)
axes[0,1].set_title('Pression Artérielle vs Cholestérol')
axes[0,1].set_xlabel('Pression artérielle repos (mmHg)')
axes[0,1].set_ylabel('Cholestérol (mg/dl)')
axes[0,1].grid(True, alpha=0.3)

# 3. Répartition par sexe et diagnostic
sex_target = pd.crosstab(df['sex'], df['target'], normalize='index') * 100
sex_target.plot(kind='bar', ax=axes[1,0], color=['lightblue', 'salmon'])
axes[1,0].set_title('Pourcentage de Maladie par Sexe')
axes[1,0].set_xlabel('Sexe (0=Femme, 1=Homme)')
axes[1,0].set_ylabel('Pourcentage (%)')
axes[1,0].legend(['Pas de maladie', 'Maladie cardiaque'])
axes[1,0].tick_params(axis='x', rotation=0)

# 4. Fréquence cardiaque maximale
box_data = [df[df['target']==0]['thalach'], df[df['target']==1]['thalach']]
bp = axes[1,1].boxplot(box_data, labels=['Pas de maladie', 'Maladie cardiaque'],
                       patch_artist=True)
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('salmon')
axes[1,1].set_title('Fréquence Cardiaque Maximale')
axes[1,1].set_ylabel('Battements par minute')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Matrice de corrélation
plt.figure(figsize=(12, 10))
corr_matrix = df.corr()

# Masquer le triangle supérieur
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Créer la heatmap
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f', cbar_kws={"shrink": .8})
plt.title('Matrice de Corrélation - Variables Cardiovasculaires', fontsize=14)
plt.tight_layout()
plt.show()

## 🔍 Analyse des Facteurs de Risque

In [ ]:
# Corrélations avec la maladie cardiaque
print("🔗 CORRÉLATIONS AVEC LA MALADIE CARDIAQUE:")
print("=" * 50)

correlations = df.corr()['target'].sort_values(key=abs, ascending=False)

for var, corr in correlations.items():
    if var != 'target':
        direction = "↗️ (positive)" if corr > 0 else "↘️ (négative)"
        if abs(corr) > 0.3:
            force = "FORTE"
        elif abs(corr) > 0.1:
            force = "modérée"
        else:
            force = "faible"
        
        print(f"{var:12} : {corr:+.3f} {direction} - corrélation {force}")
        print(f"             {descriptions[var]}")
        print()

In [ ]:
# Comparaison des moyennes entre patients sains et malades
print("📊 COMPARAISON PATIENTS SAINS vs MALADES:")
print("=" * 50)

variables_continues = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

for var in variables_continues:
    sains = df[df['target'] == 0][var]
    malades = df[df['target'] == 1][var]
    
    print(f"\n{descriptions[var]}:")
    print(f"  Patients sains    : {sains.mean():.1f} ± {sains.std():.1f}")
    print(f"  Patients malades  : {malades.mean():.1f} ± {malades.std():.1f}")
    
    difference = malades.mean() - sains.mean()
    print(f"  Différence        : {difference:+.1f}")
    
    # Test statistique simple (différence significative ?)
    from scipy import stats
    try:
        t_stat, p_value = stats.ttest_ind(malades, sains)
        significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
        print(f"  Significativité   : {significance} (p={p_value:.3f})")
    except:
        print(f"  Test statistique  : Non disponible")

## 🤖 Construction du Modèle Prédictif

In [ ]:
# Préparation des données pour le modèle
print("🔧 PRÉPARATION DES DONNÉES:")
print("=" * 30)

# Nettoyer les valeurs manquantes
df_clean = df.dropna()
print(f"Patients après nettoyage: {len(df_clean)} (supprimés: {len(df) - len(df_clean)})")

# Séparer les variables explicatives (X) de la variable cible (y)
X = df_clean.drop('target', axis=1)
y = df_clean['target']

print(f"Variables explicatives: {X.shape[1]}")
print(f"Échantillons: {X.shape[0]}")
print(f"Cas positifs (maladie): {y.sum()} ({y.mean()*100:.1f}%)")
print(f"Cas négatifs (sains): {(y==0).sum()} ({(y==0).mean()*100:.1f}%)")

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nDonnées d'entraînement: {len(X_train)} patients")
print(f"Données de test: {len(X_test)} patients")
print(f"Ratio train/test: {len(X_train)/len(X_test):.1f}:1")

In [ ]:
# Entraînement du modèle Random Forest
print("🌳 ENTRAÎNEMENT DU MODÈLE RANDOM FOREST:")
print("=" * 40)

# Créer et configurer le modèle
modele = RandomForestClassifier(
    n_estimators=100,    # Nombre d'arbres
    max_depth=10,        # Profondeur maximale
    random_state=42,     # Pour la reproductibilité
    min_samples_split=5, # Minimum d'échantillons pour diviser
    min_samples_leaf=2   # Minimum d'échantillons par feuille
)

# Entraîner le modèle
modele.fit(X_train, y_train)

print("✅ Modèle entraîné avec succès!")
print(f"Nombre d'arbres: {modele.n_estimators}")
print(f"Variables utilisées: {len(X.columns)}")

## 📈 Évaluation des Performances

In [ ]:
# Prédictions sur les ensembles d'entraînement et de test
y_pred_train = modele.predict(X_train)
y_pred_test = modele.predict(X_test)

# Calcul des métriques
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

precision_test = precision_score(y_test, y_pred_test)
recall_test = recall_score(y_test, y_pred_test)
f1_test = f1_score(y_test, y_pred_test)

print("📈 PERFORMANCES DU MODÈLE:")
print("=" * 40)
print(f"Précision entraînement: {accuracy_train:.3f} ({accuracy_train*100:.1f}%)")
print(f"Précision test:         {accuracy_test:.3f} ({accuracy_test*100:.1f}%)")
print(f"\nMétriques détaillées sur le test:")
print(f"• Précision (Precision): {precision_test:.3f} ({precision_test*100:.1f}%)")
print(f"• Sensibilité (Recall):  {recall_test:.3f} ({recall_test*100:.1f}%)")
print(f"• Score F1:              {f1_test:.3f}")

# Vérification du surapprentissage
overfitting = accuracy_train - accuracy_test
print(f"\n🎯 Analyse:")
if overfitting > 0.1:
    print(f"⚠️ Surapprentissage détecté (différence: {overfitting:.3f})")
elif overfitting < 0.05:
    print(f"✅ Bon équilibre (différence: {overfitting:.3f})")
else:
    print(f"📊 Équilibre acceptable (différence: {overfitting:.3f})")

In [ ]:
# Rapport de classification détaillé
print("📋 RAPPORT DE CLASSIFICATION DÉTAILLÉ:")
print("=" * 50)
print(classification_report(y_test, y_pred_test, 
                          target_names=['Pas de maladie', 'Maladie cardiaque']))

In [ ]:
# Matrice de confusion et métriques médicales
cm = confusion_matrix(y_test, y_pred_test)

# Visualisation de la matrice de confusion
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Pas de maladie', 'Maladie cardiaque'],
            yticklabels=['Pas de maladie', 'Maladie cardiaque'])
plt.title('Matrice de Confusion - Prédiction Maladie Cardiaque')
plt.ylabel('Vraie étiquette')
plt.xlabel('Prédiction')
plt.show()

# Calcul des métriques médicales
tn, fp, fn, tp = cm.ravel()  # True Neg, False Pos, False Neg, True Pos

sensibilite = tp / (tp + fn)  # Taux de vrais positifs (Recall)
specificite = tn / (tn + fp)  # Taux de vrais négatifs
vpp = tp / (tp + fp)          # Valeur prédictive positive (Precision)
vpn = tn / (tn + fn)          # Valeur prédictive négative

print("🏥 MÉTRIQUES MÉDICALES:")
print("=" * 30)
print(f"Sensibilité (Se):     {sensibilite:.3f} ({sensibilite*100:.1f}%)")
print(f"Spécificité (Sp):     {specificite:.3f} ({specificite*100:.1f}%)")
print(f"VPP:                  {vpp:.3f} ({vpp*100:.1f}%)")
print(f"VPN:                  {vpn:.3f} ({vpn*100:.1f}%)")

print(f"\n📊 Interprétation clinique:")
print(f"• Le modèle détecte {sensibilite*100:.0f}% des maladies cardiaques (sensibilité)")
print(f"• Il identifie correctement {specificite*100:.0f}% des patients sains (spécificité)")
print(f"• {vpp*100:.0f}% des patients prédits malades le sont vraiment (VPP)")
print(f"• {vpn*100:.0f}% des patients prédits sains le sont vraiment (VPN)")

## 🎯 Importance des Variables

In [ ]:
# Analyser l'importance des caractéristiques
importances = modele.feature_importances_
feature_importance = pd.DataFrame({
    'variable': X.columns,
    'importance': importances,
    'description': [descriptions[col] for col in X.columns]
}).sort_values('importance', ascending=False)

print("🎯 IMPORTANCE DES VARIABLES MÉDICALES:")
print("=" * 50)

for i, row in feature_importance.iterrows():
    print(f"{row['variable']:12} : {row['importance']:.3f} - {row['description']}")

# Graphique d'importance
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(10)
plt.barh(range(len(top_features)), top_features['importance'], 
         color='lightcoral', alpha=0.8)
plt.yticks(range(len(top_features)), 
          [f"{row['variable']}\n({row['description'][:20]}...)" 
           for _, row in top_features.iterrows()])
plt.xlabel('Importance')
plt.title('Importance des Variables pour la Prédiction Cardiaque')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 🔮 Prédictions sur Nouveaux Patients

In [ ]:
# Fonction pour analyser un nouveau patient
def predire_risque_cardiaque(age, sex, cp, trestbps, chol, fbs, restecg, 
                            thalach, exang, oldpeak, slope, ca, thal):
    """
    Prédire le risque de maladie cardiaque pour un nouveau patient
    
    Returns:
        dict: Prédiction et probabilité
    """
    # Créer le profil patient
    patient = np.array([[age, sex, cp, trestbps, chol, fbs, restecg,
                        thalach, exang, oldpeak, slope, ca, thal]])
    
    # Prédiction
    prediction = modele.predict(patient)[0]
    probabilites = modele.predict_proba(patient)[0]
    
    return {
        'prediction': 'Maladie cardiaque' if prediction == 1 else 'Pas de maladie',
        'probabilite_maladie': probabilites[1] * 100,
        'probabilite_sain': probabilites[0] * 100,
        'confiance': max(probabilites) * 100
    }

# Exemples de patients
print("👥 EXEMPLES DE PRÉDICTIONS:")
print("=" * 40)

# Patient 1: Homme âgé avec facteurs de risque
patient1 = predire_risque_cardiaque(
    age=65, sex=1, cp=2, trestbps=160, chol=300, fbs=1, restecg=0,
    thalach=120, exang=1, oldpeak=2.5, slope=2, ca=2, thal=3
)
print(f"👨 Patient 1 - Homme 65 ans, multiples facteurs de risque:")
print(f"   Prédiction: {patient1['prediction']}")
print(f"   Probabilité maladie: {patient1['probabilite_maladie']:.1f}%")
print(f"   Confiance: {patient1['confiance']:.1f}%")

# Patient 2: Femme jeune avec peu de facteurs
patient2 = predire_risque_cardiaque(
    age=35, sex=0, cp=0, trestbps=120, chol=200, fbs=0, restecg=0,
    thalach=180, exang=0, oldpeak=0.0, slope=0, ca=0, thal=0
)
print(f"\n👩 Patient 2 - Femme 35 ans, peu de facteurs de risque:")
print(f"   Prédiction: {patient2['prediction']}")
print(f"   Probabilité maladie: {patient2['probabilite_maladie']:.1f}%")
print(f"   Confiance: {patient2['confiance']:.1f}%")

# Patient 3: Cas intermédiaire
patient3 = predire_risque_cardiaque(
    age=50, sex=1, cp=1, trestbps=140, chol=240, fbs=0, restecg=1,
    thalach=150, exang=0, oldpeak=1.0, slope=1, ca=1, thal=2
)
print(f"\n👤 Patient 3 - Homme 50 ans, facteurs de risque modérés:")
print(f"   Prédiction: {patient3['prediction']}")
print(f"   Probabilité maladie: {patient3['probabilite_maladie']:.1f}%")
print(f"   Confiance: {patient3['confiance']:.1f}%")

## 🎯 Outil Pratique d'Analyse

In [ ]:
# Outil d'aide au diagnostic
def analyser_patient_complet(donnees_patient):
    """
    Analyser le risque cardiovasculaire d'un patient avec détails
    
    Args:
        donnees_patient (dict): Données cliniques du patient
    
    Returns:
        dict: Analyse complète avec recommandations
    """
    # Extraire les valeurs dans le bon ordre
    valeurs = [donnees_patient[col] for col in X.columns]
    patient_array = np.array([valeurs])
    
    # Prédictions
    prediction = modele.predict(patient_array)[0]
    probas = modele.predict_proba(patient_array)[0]
    
    # Analyse des facteurs de risque
    facteurs_risque = []
    
    if donnees_patient['age'] > 55:
        facteurs_risque.append("Âge > 55 ans")
    if donnees_patient['sex'] == 1:
        facteurs_risque.append("Sexe masculin")
    if donnees_patient['trestbps'] > 140:
        facteurs_risque.append("Hypertension (>140 mmHg)")
    if donnees_patient['chol'] > 240:
        facteurs_risque.append("Cholestérol élevé (>240 mg/dl)")
    if donnees_patient['fbs'] == 1:
        facteurs_risque.append("Glycémie à jeun élevée")
    if donnees_patient['exang'] == 1:
        facteurs_risque.append("Angine d'effort")
    if donnees_patient['thalach'] < 120:
        facteurs_risque.append("Fréquence cardiaque max faible")
    
    # Déterminer le niveau de risque
    prob_maladie = probas[1] * 100
    if prob_maladie > 70:
        niveau_risque = "TRÈS ÉLEVÉ"
        couleur = "🔴"
    elif prob_maladie > 50:
        niveau_risque = "ÉLEVÉ"
        couleur = "🟠"
    elif prob_maladie > 30:
        niveau_risque = "MODÉRÉ"
        couleur = "🟡"
    else:
        niveau_risque = "FAIBLE"
        couleur = "🟢"
    
    # Recommandations
    if prediction == 1:
        recommandation = "Consulter un cardiologue rapidement"
    elif prob_maladie > 30:
        recommandation = "Surveillance accrue recommandée"
    else:
        recommandation = "Suivi de routine"
    
    return {
        'risque': niveau_risque,
        'couleur': couleur,
        'probabilite': prob_maladie,
        'prediction': 'Maladie cardiaque' if prediction == 1 else 'Pas de maladie',
        'facteurs_risque': facteurs_risque,
        'nb_facteurs': len(facteurs_risque),
        'recommandation': recommandation
    }

# Exemple d'utilisation
exemple_patient = {
    'age': 60, 'sex': 1, 'cp': 2, 'trestbps': 150, 'chol': 280,
    'fbs': 1, 'restecg': 0, 'thalach': 130, 'exang': 1,
    'oldpeak': 1.5, 'slope': 2, 'ca': 1, 'thal': 3
}

resultat = analyser_patient_complet(exemple_patient)

print("💡 EXEMPLE D'ANALYSE COMPLÈTE:")
print("=" * 40)
print(f"{resultat['couleur']} Niveau de risque: {resultat['risque']}")
print(f"📊 Probabilité de maladie: {resultat['probabilite']:.1f}%")
print(f"🎯 Prédiction: {resultat['prediction']}")
print(f"📋 Nombre de facteurs de risque: {resultat['nb_facteurs']}")
print(f"⚠️ Facteurs identifiés:")
for facteur in resultat['facteurs_risque']:
    print(f"   • {facteur}")
print(f"💬 Recommandation: {resultat['recommandation']}")

## ✅ Résumé et Applications Pratiques

In [ ]:
# Résumé final de l'analyse
print("🏥 RÉSUMÉ DE L'ANALYSE CARDIOVASCULAIRE:")
print("=" * 50)
print(f"📊 Dataset analysé: {len(df)} patients, {df.shape[1]} variables")
print(f"🎯 Performance du modèle: {accuracy_test*100:.1f}% de précision")
print(f"🔬 Sensibilité: {recall_test*100:.1f}% (détection des maladies)")
print(f"🛡️ Spécificité: {specificite*100:.1f}% (identification des sains)")

print(f"\n🔍 Variables les plus importantes:")
for i, row in feature_importance.head(5).iterrows():
    print(f"   {i+1}. {row['description']} (importance: {row['importance']:.3f})")

print(f"\n💡 APPLICATIONS PRATIQUES IMMÉDIATES:")
print(f"✅ Identification précoce des patients à risque cardiovasculaire")
print(f"✅ Aide à la décision clinique pour les médecins")
print(f"✅ Priorisation des consultations cardiologiques")
print(f"✅ Stratification du risque pour la prévention")
print(f"✅ Outil de dépistage dans les centres de santé")

print(f"\n⚠️ IMPORTANTES LIMITATIONS:")
print(f"• Ce modèle est à des fins pédagogiques et de recherche")
print(f"• Il ne remplace en aucun cas l'expertise médicale")
print(f"• Validation nécessaire sur de plus grandes cohortes")
print(f"• Les résultats doivent être interprétés par un médecin")

print(f"\n🎓 COMPÉTENCES ACQUISES:")
print(f"✅ Chargement et exploration de données médicales réelles")
print(f"✅ Analyse statistique des facteurs de risque")
print(f"✅ Création de visualisations médicales professionnelles")
print(f"✅ Construction et évaluation d'un modèle prédictif")
print(f"✅ Interprétation des métriques en contexte médical")
print(f"✅ Développement d'outils d'aide au diagnostic")

print(f"\n🎯 PROCHAINE ÉTAPE:")
print(f"Dans le prochain notebook, nous analyserons des textes médicaux avec l'IA!")